In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import folium
import openpyxl
import seaborn as sns
import re
import datetime as dt
from datetime import timedelta
%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
#read in the data

members = pd.read_csv('data/NSS_GreatCare_Members.csv')
hosp_admish = pd.read_csv('data/NSS_Hospital_Inpatient_Admissions.csv')
pc_visits = pd.read_csv('data/NSS_PrimaryCare_Office_Visits.csv', low_memory=False)

In [3]:
pc_visits.head()
pc_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231373 entries, 0 to 231372
Data columns (total 18 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   Encounter ID                                   231373 non-null  object 
 1   Patient ID                                     231373 non-null  object 
 2   Service Provider ID                            230599 non-null  object 
 3   Attending Provider ID                          39644 non-null   object 
 4   Provider Group Provider ID                     227644 non-null  object 
 5   Facility Provider ID                           39756 non-null   object 
 6   Encounter Primary Payor ID                     231373 non-null  object 
 7   Encounter Primary Plan ID                      231373 non-null  object 
 8   Attending Provider NPI                         39644 non-null   float64
 9   Facility Provider NPI                

In [4]:
hosp_admish.head()
hosp_admish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Encounter ID                                         6213 non-null   object 
 1   Patient ID                                           6213 non-null   object 
 2   Encounter Primary Payor ID                           6213 non-null   object 
 3   Encounter Primary Plan ID                            6213 non-null   object 
 4   Facility Provider ID                                 6213 non-null   object 
 5   Attending Provider ID                                5884 non-null   object 
 6   Attending Provider NPI                               5884 non-null   float64
 7   Facility Provider NPI                                6213 non-null   object 
 8   Encounter Start Date                                 6213 non-null  

In [5]:
members.head()
members.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61519 entries, 0 to 61518
Data columns (total 63 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Patient ID                                                       61519 non-null  object 
 1   Primary Plan ID                                                  61519 non-null  object 
 2   Primary Payor ID                                                 61519 non-null  object 
 3   Is Patient Alive                                                 61519 non-null  int64  
 4   Patient Age Integer                                              61519 non-null  int64  
 5   Patient Date of Birth                                            61519 non-null  object 
 6   Patient ZIP Code 5-Digit                                         61105 non-null  float64
 7   Count Primary Care Visits in Last 36 Mon

### 1. What is our 30-day all cause readmission rate across hospitals our members are admitted to?

In [115]:
#look at start and end date of all hospital visits. drop irrelevant columns, count admissions for denominator (6,213)
hosp_stay = hosp_admish[['Patient ID','Encounter Start Date','Encounter End Date',
                         'Attending Provider NPI','Facility Provider NPI']]

#convert dates from objects to datetimes, convert floats to objects
hosp_stay.loc[:,['Encounter Start Date','Encounter End Date']] = hosp_stay.loc[:,['Encounter Start Date','Encounter End Date']].apply(pd.to_datetime)
hosp_stay = hosp_stay.astype({'Attending Provider NPI':object})

# hosp_stay.head()
hosp_stay.info()
#hosp_stay.count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Patient ID              6213 non-null   object        
 1   Encounter Start Date    6213 non-null   datetime64[ns]
 2   Encounter End Date      6213 non-null   datetime64[ns]
 3   Attending Provider NPI  5884 non-null   object        
 4   Facility Provider NPI   6213 non-null   object        
dtypes: datetime64[ns](2), object(3)
memory usage: 242.8+ KB


In [116]:
#if using the unique patients admitted as denominator, drop duplicate patients and count rows (4,701 unique patients)
# num_patients = hosp_stay.drop_duplicates('Patient ID')
# num_patients.count()

In [117]:
#group by patient ID and filter out any patients under 2 visits

readmish = hosp_stay.groupby('Patient ID')['Encounter Start Date'].count().reset_index()
readmish = readmish[readmish['Encounter Start Date'] >= 2]
readmish.rename({'Encounter Start Date':'# of admissions'}, axis=1)

,Patient ID,# of admissions
8,DS1|10132,2
17,DS1|10253,3
18,DS1|10291,2
32,DS1|10451,12
37,DS1|10507,2
...,...,...
4679,DS1|9809,2
4683,DS1|9864,2
4690,DS1|9935,3
4692,DS1|9943,3


In [118]:
#filter hospital stay to patients with more than one stay (from readmish table). DEBBIE'S GENIUS CODE TO CREATE NEW COLUMN
#THAT TAKES PATIENT'S NEXT START DATE AND ADDS IT TO SAME LINE AS PREVIOUS VISIT
multistay = hosp_stay[hosp_stay['Patient ID'].isin(readmish['Patient ID'])]
multistay['Next Start Date'] = multistay.sort_values('Encounter Start Date',ascending=True).groupby('Patient ID')['Encounter Start Date'].shift(-1)
multistay

<ipython-input-118-95faf4484be2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multistay['Next Start Date'] = multistay.sort_values('Encounter Start Date',ascending=True).groupby('Patient ID')['Encounter Start Date'].shift(-1)


,Patient ID,Encounter Start Date,Encounter End Date,Attending Provider NPI,Facility Provider NPI,Next Start Date
7,DS1|10132,2013-08-14,2013-08-16,NaN,1366545311,NaT
8,DS1|10132,2013-03-30,2013-04-01,1417995317.0,1386741635,2013-08-14
17,DS1|10253,2011-10-15,2011-11-13,1750346474.0,1215987532,NaT
18,DS1|10253,2010-06-02,2010-06-06,1932127248.0,1396725677,2011-10-15
19,DS1|10253,2010-04-02,2010-04-16,1346566221.0,1689772592,2010-06-02
...,...,...,...,...,...,...
6201,DS1|9943,2013-10-01,2013-10-16,1932297231.0,1184722779,2013-10-25
6202,DS1|9943,2013-10-25,2013-10-28,1942320288.0,1184722779,2014-10-28
6203,DS1|9943,2014-10-28,2014-10-30,1265649966.0,1184722779,NaT
6210,DS1|9994,2015-11-21,2015-11-23,1861831836.0,1689608150,2015-11-24


In [119]:
#subtract next start date from last end date and create new column for these results
multistay['Days Between'] = multistay['Next Start Date'] - multistay['Encounter End Date']
multistay

<ipython-input-119-2e82419d3d6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multistay['Days Between'] = multistay['Next Start Date'] - multistay['Encounter End Date']


,Patient ID,Encounter Start Date,Encounter End Date,Attending Provider NPI,Facility Provider NPI,Next Start Date,Days Between
7,DS1|10132,2013-08-14,2013-08-16,NaN,1366545311,NaT,NaT
8,DS1|10132,2013-03-30,2013-04-01,1417995317.0,1386741635,2013-08-14,135 days
17,DS1|10253,2011-10-15,2011-11-13,1750346474.0,1215987532,NaT,NaT
18,DS1|10253,2010-06-02,2010-06-06,1932127248.0,1396725677,2011-10-15,496 days
19,DS1|10253,2010-04-02,2010-04-16,1346566221.0,1689772592,2010-06-02,47 days
...,...,...,...,...,...,...,...
6201,DS1|9943,2013-10-01,2013-10-16,1932297231.0,1184722779,2013-10-25,9 days
6202,DS1|9943,2013-10-25,2013-10-28,1942320288.0,1184722779,2014-10-28,365 days
6203,DS1|9943,2014-10-28,2014-10-30,1265649966.0,1184722779,NaT,NaT
6210,DS1|9994,2015-11-21,2015-11-23,1861831836.0,1689608150,2015-11-24,1 days


In [164]:
#create another column to define whether 'Days Between' is less than 30 or not. Used Teng's for loop
pd.set_option('display.max_rows', None)

t = []

for i in multistay['Days Between']:
    if (i <= dt.timedelta(days=30)) and (i >= dt.timedelta(days=0)) :
        t.append('yes')
    else:
        t.append('no')

multistay['Under 30 Days'] = t
multistay

<ipython-input-164-959db48eb75d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multistay['Under 30 Days'] = t


,Patient ID,Encounter Start Date,Encounter End Date,Attending Provider NPI,Facility Provider NPI,Next Start Date,Days Between,Under 30 Days
7,DS1|10132,2013-08-14,2013-08-16,NaN,1366545311,NaT,NaT,no
8,DS1|10132,2013-03-30,2013-04-01,1417995317.0,1386741635,2013-08-14,135 days,no
17,DS1|10253,2011-10-15,2011-11-13,1750346474.0,1215987532,NaT,NaT,no
18,DS1|10253,2010-06-02,2010-06-06,1932127248.0,1396725677,2011-10-15,496 days,no
19,DS1|10253,2010-04-02,2010-04-16,1346566221.0,1689772592,2010-06-02,47 days,no
20,DS1|10291,2010-05-14,2010-05-15,1598993990.0,1821250762,2011-02-24,285 days,no
21,DS1|10291,2011-02-24,2011-03-05,1720240526.0,1902803315,NaT,NaT,no
34,DS1|10451,2014-04-24,2014-04-26,1306902739.0,1023010113,2014-05-23,27 days,yes
35,DS1|10451,2013-03-20,2013-03-23,1518963032.0,1063412005,2013-03-27,4 days,yes
36,DS1|10451,2014-06-29,2014-06-29,1750425278.0,1225199987,NaT,NaT,no


In [165]:
#Why doesn't this work?
# thirty_dict = {dt.timedelta(days<=30):'yes',dt.timedelta(days>30):'no'}

# multistay['Under 30 Days'] = multistay['Days Between'].map(thirty_dict)
# multistay

In [166]:
#count yes, use as numerator to calculate rate of <30 Day Readmission
multistay['Under 30 Days'].value_counts()

no     1686
yes     696
Name: Under 30 Days, dtype: int64

In [168]:
rate_under_30days = round((696/6213)*100,2)
rate_under_30days

11.2

### 2. What percent of patients have a PCP visit within a week of discharge?

In [135]:
pc_visits.head()

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code
0,COV|EE|DS1|10076|20110225|DS1|NPI|1487856779,DS1|10076,DS1|NPI|1487856779,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.487857e+09,1.043427e+09,2011-02-18,0,1,0,0,NaN
1,COV|EE|DS1|10088|20140222|DS1|NPI|1528030996,DS1|10088,DS1|NPI|1528030996,DS1|NPI|1528030996,DS1|NPI|1043427248,DS1|NPI|1902803315,DS1|1,DS1|5047W,1.528031e+09,1.902803e+09,1.528031e+09,1.043427e+09,2014-02-22,0,1,1,1,NaN
2,COV|EE|DS1|10165|20140907|DS1|NPI|1477579365,DS1|10165,DS1|NPI|1477579365,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.477579e+09,1.043427e+09,2014-08-31,0,1,0,0,NaN
3,COV|EE|DS1|10240|20130624|DS1|NPI|1821178088,DS1|10240,DS1|NPI|1821178088,DS1|NPI|1821178088,DS1|NPI|1407058787,DS1|NPI|1104842020,DS1|1,DS1|5047W,1.821178e+09,1.104842e+09,1.821178e+09,1.407059e+09,2013-06-17,0,1,1,1,NaN
4,COV|EE|DS1|10293|20160904|DS1|NPI|1609004191,DS1|10293,DS1|NPI|1609004191,NaN,DS1|NPI|1184722779,NaN,DS1|1,DS1|5047X,NaN,NaN,1.609004e+09,1.184723e+09,2016-09-04,0,1,0,0,NaN


In [144]:
#limit columns, convert relevant ones to objects and Encounter Date to datetime
pcp_trips = pc_visits[['Patient ID', 'Attending Provider NPI', 'Facility Provider NPI', 'Provider Group NPI', 'Encounter Date']]
pcp_trips = pcp_trips.astype(object)
pcp_trips['Encounter Date'] = pd.to_datetime(pcp_trips['Encounter Date'])
pcp_trips = pcp_trips.rename({'Encounter Date':'PCP Date'}, axis=1)
pcp_trips


,Patient ID,Attending Provider NPI,Facility Provider NPI,Provider Group NPI,PCP Date
0,DS1|10076,NaN,NaN,1043427248.0,2011-02-18
1,DS1|10088,1528030996.0,1902803315.0,1043427248.0,2014-02-22
2,DS1|10165,NaN,NaN,1043427248.0,2014-08-31
3,DS1|10240,1821178088.0,1104842020.0,1407058787.0,2013-06-17
4,DS1|10293,NaN,NaN,1184722779.0,2016-09-04
...,...,...,...,...,...
231368,DS1|9999,NaN,NaN,1184722779.0,2015-02-10
231369,DS1|99,NaN,NaN,1043427248.0,2011-08-02
231370,DS1|99,NaN,NaN,1043427248.0,2011-04-13
231371,DS1|9,NaN,NaN,1962790014.0,2014-04-07


In [155]:
#merge hospital stays and pcp visits on patient ID; there are many duplicates (for each dr visit)
hosp_and_pcp_dates = hosp_stay.merge(pcp_trips, on='Patient ID')
#hosp_and_pcp_dates = hosp_and_pcp_dates.drop_duplicates(subset=['Encounter Start Date','Encounter End Date'])
hosp_and_pcp_dates = hosp_and_pcp_dates.drop(columns=['Attending Provider NPI_y','Facility Provider NPI_y'])
hosp_and_pcp_dates

,Patient ID,Encounter Start Date,Encounter End Date,Attending Provider NPI_x,Facility Provider NPI_x,Provider Group NPI,PCP Date
0,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2015-12-08
1,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2012-06-18
2,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-06-10
3,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-09-28
4,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-03-16
...,...,...,...,...,...,...,...
43786,DS1|9994,2015-11-24,2015-11-27,1750544516.0,1689608150,1407058787.0,2015-06-22
43787,DS1|9994,2015-11-24,2015-11-27,1750544516.0,1689608150,1407058787.0,2015-12-05
43788,DS1|9994,2015-11-24,2015-11-27,1750544516.0,1689608150,1407058787.0,2015-12-13
43789,DS1|9994,2015-11-24,2015-11-27,1750544516.0,1689608150,1205884392.0,2014-05-18


In [159]:
hosp_and_pcp_dates['Days Between'] = hosp_and_pcp_dates['PCP Date'] - hosp_and_pcp_dates['Encounter End Date']
#hosp_and_pcp_dates

#create another column to define whether 'Days Between' is less than 30 or not. Used Teng's for loop
x = []

for i in hosp_and_pcp_dates['Days Between']:
    if (i <= dt.timedelta(days=7)) and (i > dt.timedelta(days=0)) :
        x.append('yes')
    else:
        x.append('no')

hosp_and_pcp_dates['Under 7 Days'] = x
hosp_and_pcp_dates.head(30)


,Patient ID,Encounter Start Date,Encounter End Date,Attending Provider NPI_x,Facility Provider NPI_x,Provider Group NPI,PCP Date,Days Between,Under 7 Days
0,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2015-12-08,1058 days,no
1,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2012-06-18,-210 days,no
2,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-06-10,512 days,no
3,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-09-28,622 days,no
4,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-03-16,426 days,no
5,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-12-09,694 days,no
6,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2015-11-08,1028 days,no
7,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2012-05-18,-241 days,no
8,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-05-10,481 days,no
9,DS1|10033,2013-01-12,2013-01-14,1417170523.0,1306890389,1043427248.0,2014-08-28,591 days,no


In [158]:
hosp_and_pcp_dates['Under 7 Days'].value_counts()

no     41429
yes     2362
Name: Under 7 Days, dtype: int64

In [161]:
pcp_trips.value_counts('Patient ID')

Patient ID
DS1|44027    94
DS1|13892    81
DS1|51224    59
DS1|32475    57
DS1|29329    56
             ..
DS1|52872     1
DS1|26674     1
DS1|52875     1
DS1|52881     1
DS1|37672     1
Length: 50734, dtype: int64


### 3. Do either vary by individual or group PCP's?

### Bonus:
### 4. Which individual or PCP groups should we make an extra effort with to build a relationship? Which have the highest volume

### 5. What percent of our patients do not have a PCP assigned or haven't seen their PCP recently?

### 6. Are there certain patients we should focus on (e.g. frequent fliers)?

### 7. From this, what could a version 1 of an actionable dashboard or intervention list look like to support the Transition Care Managers?